<a href="https://colab.research.google.com/github/alibekk93/IDP_analysis/blob/RAPID/getting_proteomes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting UniProt proteomes for Tempura species

## Setup

In [49]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [187]:
### Read fasta ###

def read_fasta(fasta_file: str) -> pd.DataFrame:
  """Processes raw .fasta files

  Opens a .fasta file, parses the sequences and their IDs into a
  dataframe and returns the dataframe.

  Parameters
  ----------
  fasta_file : str
    the raw .fasta file directory.

  Returns
  -------
  pd.DataFrame
    a dataframe with ID, Sequence and Length columns.

  """

  # open the file
  handle = open(fasta_file, 'r')
  seq_list = list(SeqIO.parse(handle, 'fasta'))
  handle.close()

  # parse data into lists
  ids = [seq_record.id.split('|')[1] for seq_record in seq_list]
  seqs = [str(seq_record.seq) for seq_record in seq_list]
  lens = [len(seq) for seq in seqs]

  # save data into a dataframe
  df = pd.DataFrame({'ID':ids, 'Sequence':seqs, 'Length':lens})

  return df

Loading Tempura dataset

In [140]:
# tempura = pd.read_csv('/content/200617_TEMPURA.csv')
tempura = pd.read_csv('/content/tempura_bacteria_uniprot.csv', index_col=0)

Only keeping bacteria with available assembly or accession numbers

In [141]:
tempura = tempura[tempura['superkingdom']=='Bacteria']
tempura.dropna(subset='assembly_or_accession', inplace=True)
tempura.reset_index(drop=True, inplace=True)

## Getting UniProt IDs

Tepura has NCBI taxonomy IDs, but we need UniProt proteome IDs. We can get them using UniProt REST API search

In [50]:
uniprot_jsons = []
failures = []

# loop through the taxonomy IDs and retrieve proteome data
for tax_id in tqdm(tempura['taxonomy_id']):
  # define the UniProt API URL
  url = f'https://rest.uniprot.org/proteomes/stream?format=json&query=%28%28taxonomy_id%3A{tax_id}%29%29'
  # send an HTTP GET request to the UniProt API
  response = requests.get(url)
  # Check if the request was successful
  if response.status_code == 200:
    # save JSON
    uniprot_jsons.append(response.json())
  else:
    failures.append(tax_id)
    uniprot_jsons.append({})

100%|██████████| 893/893 [08:24<00:00,  1.77it/s]


In many cases we get mre than one search result for one taxonomy ID. We need to check each search result and only keep the UniProt ID that has the same taxonomy ID as Tempura

In [91]:
# initiate empty list to save UniProt IDs
uniprot_ids = []

# iterate through JSON results
for i, jsn in enumerate(uniprot_jsons):
  # get results
  results = jsn['results']
  # get candidate UniProt IDs and corresponding taxonomy IDs
  u_ids = [r['id'] for r in results]
  t_ids = [r['taxonomy']['taxonId'] for r in results]
  # make a dictionary of candidate IDs
  results_dict = {k:v for k, v in zip(t_ids, u_ids)}
  # get actual taxonomy ID
  taxonomy_id = tempura.loc[i, 'taxonomy_id']
  # save correct UniProt ID
  try:
    uniprot_ids.append(results_dict[taxonomy_id])
  except:
    # no correct ID found
    uniprot_ids.append(None)

We can now drop any Tempura rows with no available UniProt IDs and store the result

In [103]:
tempura['uniprot_id'] = uniprot_ids

In [115]:
tempura.dropna(subset='uniprot_id', inplace=True)

In [117]:
tempura.reset_index(drop=True, inplace=True)

In [137]:
tempura.to_csv('tempura_bacteria_uniprot.csv')

## Downloading UniProt proteomes

In [143]:
!mkdir proteomes

In [146]:
tempura.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 680 entries, 0 to 679
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   genus_and_species      680 non-null    object 
 1   taxonomy_id            680 non-null    int64  
 2   strain                 680 non-null    object 
 3   superkingdom           680 non-null    object 
 4   phylum                 680 non-null    object 
 5   class                  678 non-null    object 
 6   order                  673 non-null    object 
 7   family                 662 non-null    object 
 8   genus                  675 non-null    object 
 9   assembly_or_accession  680 non-null    object 
 10  Genome_GC              651 non-null    float64
 11  Genome_size            680 non-null    float64
 12  16S_accssion           680 non-null    object 
 13  16S_GC                 680 non-null    float64
 14  Tmin                   680 non-null    float64
 15  Topt_a

In [165]:
failures = []

for i in tqdm(tempura.index):
  # make file path and get UniProt ID
  species = tempura.loc[i, 'genus_and_species'].replace(' ', '_')
  fasta_file_path = f'/content/proteomes/{species}.fasta'
  id = tempura.loc[i, 'uniprot_id']
  # define the UniProt API URL to retrieve FASTA data
  url = f'https://rest.uniprot.org/uniprotkb/stream?format=fasta&query=%28%28proteome%3A{id}%29%29'
  # send an HTTP GET request to the UniProt API to get FASTA data
  response = requests.get(url)
  # check if the request was successful
  if response.status_code == 200:
    # save the FASTA data to a file
    with open(fasta_file_path, 'w') as fasta_file:
      fasta_file.write(response.text)
  else:
    failures.append(id)

100%|██████████| 680/680 [56:33<00:00,  4.99s/it]


In [166]:
failures

[]

In [ ]:
!zip -r /content/proteomes.zip /content/proteomes -i '*.fasta'
from google.colab import files
files.download('/content/proteomes.zip')

## Creating a DataFrame with all sequences

In [202]:
all_proteins = pd.DataFrame(columns=['ID', 'Sequence', 'Length', 'Species'])

In [203]:
for species in tqdm(tempura['genus_and_species']):
  filename = species.replace(' ', '_') + '.fasta'
  df = read_fasta(f'/content/proteomes/{filename}')
  df['Species'] = species
  all_proteins = pd.concat([all_proteins, df], ignore_index=True)

100%|██████████| 680/680 [01:02<00:00, 10.87it/s]


In [205]:
all_proteins.to_csv('all_proteins.csv')

Filtering to only keep species with at least 1000 proteins

In [231]:
# group the DataFrame by the 'species' column and count the number of records for each species
species_counts = all_proteins['Species'].value_counts()
# filter the species with more than 1000 records
selected_species = species_counts[species_counts >= 1000].index
# create a new DataFrame that only includes the selected species
all_proteins_filtered = all_proteins[all_proteins['Species'].isin(selected_species)].reset_index(drop=True)

Remove species that don't have 1000 records from Tempura

In [238]:
tempura_filtered = tempura[tempura['genus_and_species'].isin(selected_species)].reset_index(drop=True)

In [241]:
tempura_filtered.to_csv('tempura_filtered.csv')

Saving combined FASTA files for disorder calculations

In [259]:
!mkdir /content/combined_fastas

In [260]:
# define the maximum sequences per file
max_sequences_per_file = 75000

# split the DataFrame into chunks of max_sequences_per_file and save as FASTA files
for i, chunk in enumerate(range(0, len(all_proteins_filtered), max_sequences_per_file)):
  chunk_df = all_proteins_filtered.iloc[chunk:chunk + max_sequences_per_file]
  # create a FASTA file for the chunk
  fasta_file_path = f'/content/combined_fastas/output_{i+1}.fasta'
  with open(fasta_file_path, 'w') as fasta_file:
    for _, row in chunk_df.iterrows():
      id = row['ID']
      seq = row['Sequence']
      fasta_file.write(f'>{id}\n{seq}\n')

In [261]:
!zip -r /content/combined_fastas.zip /content/combined_fastas -i '*.fasta'
from google.colab import files
files.download('/content/combined_fastas.zip')

  adding: content/combined_fastas/output_9.fasta (deflated 43%)
  adding: content/combined_fastas/output_12.fasta (deflated 43%)
  adding: content/combined_fastas/output_10.fasta (deflated 43%)
  adding: content/combined_fastas/output_4.fasta (deflated 43%)
  adding: content/combined_fastas/output_2.fasta (deflated 43%)
  adding: content/combined_fastas/output_5.fasta (deflated 43%)
  adding: content/combined_fastas/output_3.fasta (deflated 43%)
  adding: content/combined_fastas/output_8.fasta (deflated 43%)
  adding: content/combined_fastas/output_1.fasta (deflated 43%)
  adding: content/combined_fastas/output_6.fasta (deflated 43%)
  adding: content/combined_fastas/output_16.fasta (deflated 43%)
  adding: content/combined_fastas/output_7.fasta (deflated 43%)
  adding: content/combined_fastas/output_13.fasta (deflated 43%)
  adding: content/combined_fastas/output_17.fasta (deflated 43%)
  adding: content/combined_fastas/output_11.fasta (deflated 43%)
  adding: content/combined_fastas/

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [51]:
failures = []
species = []

for id in tempura.loc[10:30, 'taxonomy_id']:
  try:
    Entrez.email = 'arkug@uottawa.ca'
    taxonomy_id = id
    handle = Entrez.efetch(db="taxonomy", id=taxonomy_id, retmode="xml")
    record = Entrez.read(handle)
    organism_name = record[0]["ScientificName"].replace(' ', '_')
    species.append(organism_name)
    handle.close()
    handle = Entrez.esearch(db='protein', term=f'txid{taxonomy_id}[Organism]', retmax=100000)
    record = Entrez.read(handle)
    handle.close()
    id_list = record['IdList']
    handle = Entrez.efetch(db='protein', id=id_list, rettype='fasta', retmode='text')
    records = SeqIO.parse(handle, 'fasta')
    SeqIO.write(records, f'{organism_name}.fasta', 'fasta')
    handle.close()
  except:
    failures.append(id)
    pass

In [67]:
taxonomy_id = '526'
url = f'https://rest.uniprot.org/proteomes/search?compressed=true&format=list&query=%28%28taxonomy_id%3A1609559%29%29&size=5'
response = requests.get(url)

In [70]:
response.content

b'\x1f\x8b\x08\x00\x00\x00\x00\x00\x00\x00\x0b\r0\x00\x02s\x03S\x0bs.\x00\x90\xf8\xde\xe9\x0c\x00\x00\x00'

In [68]:
with open(f"{taxonomy_id}.txt", "wb") as f:
    f.write(response.content)

In [18]:
import requests

# List of taxonomy IDs
taxonomy_ids = [562, 1609559]  # Add your taxonomy IDs here

# Initialize an empty DataFrame to store the proteome data
proteome_data = pd.DataFrame(columns=['Entry', 'Entry name', 'Protein names', 'Gene names', 'Organism'])

# Loop through the taxonomy IDs and retrieve proteome data
for tax_id in taxonomy_ids:
    # Define the UniProt API URL
    url = f'https://rest.uniprot.org/proteomes/stream?format=json&query=%28%28taxonomy_id%3A{tax_id}%29%29'

    # Send an HTTP GET request to the UniProt API
    response = requests.get(url)

#     # Check if the request was successful
#     if response.status_code == 200:
#         # Parse the response text and append it to the DataFrame
#         data = response.text.strip().split('\n')
#         data = [line.split('\t') for line in data]
#         proteome_data = proteome_data.append(pd.DataFrame(data, columns=proteome_data.columns))
#     else:
#         print(f"Failed to retrieve data for taxonomy ID {tax_id}")

# # Save the proteome data to a CSV file
# proteome_data.to_csv('proteome_data.csv', index=False)


In [30]:
json = response.json()

In [34]:
json['results'][0]['id']

'UP000070587'

In [37]:
json['results'][0]['taxonomy']['scientificName']

'Pyrococcus kukulkanii'

In [42]:
json['results'][0]

{'id': 'UP000070587',
 'taxonomy': {'scientificName': 'Pyrococcus kukulkanii', 'taxonId': 1609559},
 'modified': '2022-06-23',
 'proteomeType': 'Other proteome',
 'strain': 'NCB100',
 'components': [{'name': 'Chromosome',
   'description': 'Pyrococcus sp. NCB100',
   'proteinCount': 1997,
   'genomeAnnotation': {'source': 'ENA/EMBL'},
   'proteomeCrossReferences': [{'database': 'GenomeAccession',
     'id': 'CP010835'},
    {'database': 'Biosample', 'id': 'SAMN03323778'}]}],
 'citations': [{'id': '27189596',
   'citationType': 'journal article',
   'authors': ['Callac N.',
    'Oger P.',
    'Lesongeur F.',
    'Rattray J.E.',
    'Vannier P.',
    'Michoud G.',
    'Beauverger M.',
    'Gayet N.',
    'Rouxel O.',
    'Jebbar M.',
    'Godfroy A.'],
   'citationCrossReferences': [{'database': 'PubMed', 'id': '27189596'},
    {'database': 'DOI', 'id': '10.1099/ijsem.0.001160'}],
   'title': 'Pyrococcus kukulkanii sp. nov., a hyperthermophilic, piezophilic archaeon isolated from a deep-

In [52]:
failures

[2265, 1104324, 985052, 229980, 2180, 1273541, 94694, 2271, 187880, 187878]

In [50]:
!zip -r /content/fasta_files.zip /content -i '*.fasta'
from google.colab import files
files.download('/content/fasta_files.zip')

  adding: content/Pyrobaculum_aerophilum.fasta (deflated 53%)
  adding: content/Pyrococcus_furiosus.fasta (deflated 54%)
  adding: content/Pyrolobus_fumarii.fasta (deflated 48%)
  adding: content/Pyrococcus_yayanosii.fasta (deflated 48%)
  adding: content/Pyrobaculum_islandicum.fasta (deflated 50%)
  adding: content/Pyrococcus_abyssi.fasta (deflated 51%)
  adding: content/Hyperthermus_butylicus.fasta (deflated 49%)
  adding: content/Pyrococcus_horikoshii.fasta (deflated 59%)
  adding: content/Pyrococcus_kukulkanii.fasta (deflated 48%)
  adding: content/Escherichia_coli.fasta (deflated 45%)
  adding: content/Thermoproteus_uzoniensis.fasta (deflated 50%)
  adding: content/Methanopyrus_kandleri.fasta (deflated 52%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>